# The Outer Solar System

---


## Data

The chosen units are: masses relative to the sun, so that the sun has mass $1$. We have taken $m_0 = 1.00000597682$ to take account of the inner planets. Distances are in astronomical units , times in earth days, and the gravitational constant is thus $G = 2.95912208286 \cdot 10^{−4}$.

| planet | mass | initial position | initial velocity |
| --- | --- | --- | --- |
| Jupiter | $m_1 = 0.000954786104043$ | <ul><li>−3.5023653</li><li>−3.8169847</li><li>−1.5507963</li></ul> | <ul><li>0.00565429</li><li>−0.00412490</li><li>−0.00190589</li></ul>
| Saturn | $m_2 = 0.000285583733151$ | <ul><li>9.0755314</li><li>−3.0458353</li><li>−1.6483708</li></ul> | <ul><li>0.00168318</li><li>0.00483525</li><li>0.00192462</li></ul>
| Uranus | $m_3 = 0.0000437273164546$ | <ul><li>8.3101420</li><li>−16.2901086</li><li>−7.2521278</li></ul> | <ul><li>0.00354178</li><li>0.00137102</li><li>0.00055029</li></ul>
| Neptune | $m_4 = 0.0000517759138449$ | <ul><li>11.4707666</li><li>−25.7294829</li><li>−10.8169456</li></ul> | <ul><li>0.00288930</li><li>0.00114527</li><li>0.00039677</li></ul>
| Pluto | $ m_5 = 1/(1.3 \cdot 10^8 )$ | <ul><li>−15.5387357</li><li>−25.2225594</li><li>−3.1902382</li></ul> | <ul><li>0.00276725</li><li>−0.00170702</li><li>−0.00136504</li></ul>

The data is taken from the book "Geometric Numerical Integration" by E. Hairer, C. Lubich and G. Wanner.

In [17]:
using Plots, DiffEqPhysics, ForwardDiff
gr()

Plots.GRBackend()

In [6]:
const G = 2.95912208286e-4
M = Diagonal([1.00000597682, 0.000954786104043, 0.000285583733151,
        0.0000437273164546, 0.0000517759138449, 1/1.3e8])       # repeat each mass three time
                                                                          # becasue there are 3 degrees
                                                                          # of freedom for each planet/sun

planets = ["Sun", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"]

pos = [
    0.,0.,0.,                              -3.5023653,-3.8169847,-1.5507963,
    9.0755314,-3.0458353,-1.6483708,       8.3101420,-16.2901086,-7.2521278,
    11.4707666,-25.7294829,-10.8169456,    -15.5387357,-25.2225594,-3.1902382
]
vel = [
    0,0,0,                               0.00565429,-0.00412490,-0.00190589,
    0.00168318,0.00483525,0.00192462,    0.00354178,0.00137102,0.00055029,
    0.00288930,0.00114527,0.00039677,    0.00276725, -0.00170702, -0.00136504
]

tspan = (0.,200_000)

(0.0, 200000)

The N-body problem's Hamiltonian is

$$H(p,q) = \frac{1}{2}\sum_{i=0}^{N}\frac{p_{i}^{T}p_{i}}{m_{i}} - G\sum_{i=1}^{N}\sum_{j=0}^{i-1}\frac{m_{i}m_{j}}{\left\lVert q_{i}-q_{j} \right\rVert}$$

Here, we want to solve for the motion of the five outer planets relative to the sun, namely, Jupiter, Saturn, Uranus, Neptune and Pluto.

In [23]:
const ∑ = sum
N = 5

ø = i -> (3i+1):(3(i+1))    # Generate indices like 1:3, 4:6 ...
T(p) = ∑(i->(p[ø(i)]'p[ø(i)])/M[3i+1,3i+1], 0:N)/2
V(t,x,y,z,M) = -G*∑(i->∑(j->(M[i+1,i+1]*M[j+1,j+1])
                    /sqrt((x[i+1]-x[j+1])^2 + (y[i+1]-y[j+1])^2 + (z[i+1]-z[j+1])^2),
                0:i-1), 1:N)
V(t,q) = -G*∑(i->∑(j->(M[i+1,i+1]*M[j+1,j+1])
                    /norm(q[i]-q[j]),
                0:i-1), 1:N)
function Hamiltonian(p,q)
    T(p) + V(q)
    T+V
end

Hamiltonian (generic function with 1 method)

## Hamiltonian System

`NBodyProblem` constructs a second order ODE problem under the hood. We know that a Hamiltonian system has the form of

$$\dot{p} = -H_{q}(p,q)\quad \dot{q}=H_{p}(p,q)$$

We can symplify $\dot{q}$ as

$$\dot{p} = -\nabla{V}(q)\quad \dot{q}=M^{-1}p$$

because only potential energy depends on $q$ in a N-body system. So, we can write a second order ODE as

$$\ddot{q} = f(q) = -M^{-1}\nabla{V}(q)$$

In [19]:
prob = NBodyProblem(V, M, pos, vel, tspan)
@time sol = solve(prob, Yoshida6(), dt=100);

  0.373751 seconds (2.02 M allocations: 92.442 MiB, 5.66% gc time)


In [21]:
plot_orbits(sol, body_names=planets)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 131.496,274.086 128.133,276.992 131.496,274.086 131.496,258.97 129.815,260.423 131.496,258.97 131.496,243.854 128.133,246.759 131.496,243.854 
 131.496,228.738 129.815,230.19 131.496,228.738 131.496,213.621 128.133,216.527 131.496,213.621 131.496,198.505 129.815,199.958 131.496,198.505 131.496,183.389 
 128.133,186.295 131.496,183.389 131.496,168.273 129.815,169.726 131.496,168.273 131.496,153.157 128.133,156.062 131.496,153.157 
 "/>
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 131.496,274.086 131.951,274.318 130.27,275.771 131.951,274.318 158.672,287.96 155.31,290.866 158.672,287.96 185.394,301.601 183.712,303.054 185.394,301.601 
 212.115,315.243 208.752,318.149 212.115,315.243 238.836,328.884 237.155,330.337 238.836,328.884 265.558,342.526 262.195,345.432 265.558,342.526 292.279,356.167 
 290.598,357.62 292.279,356.167 319,369.809 315.638,372.715 319,369.809 331.868,376.378 
 "/>
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 331.868,376.378 333.23,375.201 335.234,376.224 333.23,375.201 355.644,355.833 359.651,357.879 355.644,355.833 378.057,336.466 380.061,337.489 378.057,336.466 
 400.471,317.098 404.478,319.144 400.471,317.098 422.884,297.731 424.888,298.754 422.884,297.731 445.298,278.363 449.305,280.409 445.298,278.363 467.711,258.996 
 469.715,260.019 467.711,258.996 490.125,239.628 494.132,241.674 490.125,239.628 500,231.095 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 
 280.718,195.499 280.718,195.499 280.718,195.499 280.718,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 
 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.499 280.717,195.5 
 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 280.716,195.5 
 280.716,195.5 280.715,195.5 280.715,195.501 280.715,195.501 280.715,195.501 280.715,195.501 280.715,195.501 280.715,195.501 280.715,195.501 280.714,195.501 
 280.714,195.501 280.714,195.502 280.714,195.502 280.714,195.502 280.714,195.502 280.714,195.502 280.713,195.502 280.713,195.502 280.713,195.502 280.713,195.503 
 280.713,195.503 280.713,195.503 280.713,195.503 280.712,195.503 280.712,195.503 280.712,195.504 280.712,195.504 280.712,195.504 280.712,195.504 280.712,195.504 
 280.711,195.504 280.711,195.505 280.711,195.505 280.711,195.505 280.711,195.505 280.711,195.505 280.71,195.506 280.71,195.506 280.71,195.506 280.71,195.506 
 280.71,195.507 280.71,195.507 280.709,195.507 280.709,195.507 280.709,195.507 280.709,195.508 280.709,195.508 280.709,195.508 280.708,195.508 280.708,195.509 
 280.708,195.509 280.708,195.509 280.708,195.51 280.708,195.51 280.707,195.51 280.707,195.51 280.707,195.511 280.707,195.511 280.707,195.511 280.706,195.511 
 280.706,195.512 280.706,195.512 280.706,195.512 280.706,195.513 280.706,195.513 280.705,195.513 280.705,195.514 280.705,195.514 280.705,195.514 280.705,195.515 
 280.705,195.515 280.704,195.515 280.704,195.516 280.704,195.516 280.704,195.516 280.704,195.517 280.704,195.517 280.704,195.517 280.703,195.518 280.703,195.518 
 280.703,195.518 280.703,195.519 280.703,195.519 280.703,195.519 280.703,195.52 280.702,195.52 280.702,195.521 280.702,195.521